# Reinfocement Learning and Causality - Advances



__Purpose of this notebook__
- Whole set of innovative papers developed in the last two years
- Show two papers as an example, overview of the remaining papers 

## Liu et al. (2018) Representation Balancing MDPs for Off-Policy Policy Evaluation

* Consider how to estimate individual policy value and average policy value
* Explicitly inspired by literature on causal inference
* Individual policy value (ITE), average policy (ATE)



**Paper**, [here](https://papers.nips.cc/paper/2018/file/980ecd059122ce2e50136bda65c25e07-Paper.pdf). 


__Motivation:__
* Off policy evaluation where deploying new policy might be costly or risky 
* E.g. consumer marketing, healthcare and education
* Two sources of difference: (1) behavioural policy may select different action than preferred by evaluation policy (2) future states get defined by current actions
* Distribution of data under behavioural policy different than under desired evaluation policy
* Every action not selected in reinforcement learning or mutli-armed bandits is a counterfactual

__Background__
* Approaches for off policy evaluation must make a choice whether and how to address data distribution mismatch
* _One way - model free:_ Importance sampling - unbiased and strongly  consistent but high variance [inefficient use of data]
* And any variance made exponentially increases due to the sequential nature of the decision making (multiply the individual errors)
* Given that we have full knowledge of the behavioural policy (otherwise strongly biased) -> rarely the case in real-world situations
* _Another way - model based:_ ignore mismatch and and fit maximum likelihood model of rewards and dynamics from behavioural data (basically any model that is fit)
* Lower variance but biased (do not converge to the true estimate of the policy's value)
* Nevertheless often achieve better empirical performance than IS-based estimators


__Aim:__

* Model based estimator of off policy policy evaluation that _do_ have theoretical convergene guarantees (unbiased)
* Also yield better empirical performance than model-based approach that ignore the distributional mismatch
* And provide finite sample generalization error!!

Typically in RL policy evaluated in terms of Average policy value:


This is the ATE applied to RL! E.g. how on average a policy performs for patients 


$$
\left[\mathbb{E}_{s_{0}} \widehat{V}^{\pi_{e}}\left(s_{0}\right)-\mathbb{E}_{s_{0}} V^{\pi_{e}}\left(s_{0}\right)\right]^{2}
$$

More appropriate though Individual Policy Value 

$$
\mathbb{E}_{s_{0}}\left[\widehat{V}^{\pi_{e}}\left(s_{0}\right)-V^{\pi_{e}}\left(s_{0}\right)\right]^{2}
$$


This is the ITE in RL! E.g. How a policy performs for a particular Patient






__How is this done?__ 

by learning a model to predict
individual outcomes using a (model fitting) loss function that explicitly accounts for the data distribution shift between the treatment and control policies

And adding a term that measures the difference between the original and the new distribution (we have seen this multiple times in causality before)

__Overall Loss function__

$$
\mathcal{L}\left(\widehat{M}_{\phi} ; \alpha_{t}\right)=\widehat{R}_{\mu}\left(\widehat{M}_{\phi}\right)+\widehat{R}_{\pi, u}\left(\widehat{M}_{\phi}\right)+\sum_{t=0}^{H-1} \alpha_{t} \operatorname{IPM}_{\mathcal{F}}\left(\widehat{p}_{M, \mu}^{\phi, F}\left(z_{t}\right), \widehat{p}_{M, \mu}^{\phi, C F}\left(z_{t}\right)\right)+\frac{\mathfrak{R}\left(\widehat{M}_{\phi}\right)}{n^{3 / 8}}
$$

where:

* $\mu$ stands for the proposed policy


* $\pi$ stands for the existing policy 


* $\widehat{M}_{\phi}$ represent the estimated model


* $ \operatorname{IPM}$ measures the difference between the observed counterfactual (inferred distribution/ policy) in terms of MMD- Mean Maximum Discrepancy, but other distances like Wasserstein or total variational distance can also be used


* $\Re\left(\widehat{M}_{\phi}\right)$ stands for an additional regularization term related to the model


* $\phi(s)$ and $\widehat{M}_{\phi}$ are parameterized by neural networks, due to their strong ability to learn representations.


* and where $\widehat{R}$ are the propensity weighted loss of the model in terms of two components a transition loss and a reward loss, that has been weighted with a propensity score

 The propensity weighted loss is based on which represents the first two "R" terms in the above equation is based on the last term in theorem 1:
 
 Theorem 1. For any MDP $M$, approximate MDP model $\widehat{M}$, behavior policy $\mu$ and deterministic evaluation policy $\pi,$ let $B_{\phi, t}$ and $G_{t}$ be a real number and function family that satisfy the condition in Lemma 4 Then:
$$
\begin{aligned}
\mathbb{E}_{s_{0}}\left[V_{\widehat{M}}^{\pi}\left(s_{0}\right)\right.&\left.-V_{M}^{\pi}\left(s_{0}\right)\right]^{2} \leq 2 H \sum_{t=0}^{H-1}\left[B_{\phi, t} I P M_{G_{t}}\left(p_{M, \mu}^{\phi, F}\left(z_{t}\right), p_{M, \mu}^{\phi, C F}\left(z_{t}\right)\right)\right.\\
&\left.+\int_{\mathcal{S}} \frac{1}{p_{M, \mu}\left(a_{0: t}=\pi\right)}\left(\bar{\ell}_{r}\left(s_{t}, \pi\left(s_{t}\right), \widehat{M}\right)+\bar{\ell}_{T}\left(s_{t}, \pi\left(s_{t}\right), \widehat{M}\right)\right) p_{M, \mu}\left(s_{t}, a_{0: t}=\pi\right) d s_{t}\right]
\end{aligned}
$$
 
, where $H$ represents the horizon, which is < $\infty$
 

And where the total loss can be decomposed into the loss of estimating the reward and the loss of estimating the transition likelihoods.

The square error loss function of value function, reward, transition are:
$$
\begin{array}{c}
\left.\bar{\ell}_{V}(s, \widehat{M}, H-t)=\left(V \frac{\pi}{M}, H-t^{(}\right)-V_{M, H-t}^{\pi}(s)\right)^{2} \quad \bar{\ell}_{r}\left(s_{t}, a_{t}, \widehat{M}\right)=\left(\widehat{r}\left(s_{t}, a_{t}\right)-\bar{r}\left(s_{t}, a_{t}\right)\right)^{2} \\
\bar{\ell}_{T}\left(s_{t}, a_{t}, \widehat{M}\right)=\left(\int_{\mathcal{S}}\left(\widehat{T}\left(s^{\prime} \mid s_{t}, a_{t}\right)-T\left(s^{\prime} \mid s_{t}, a_{t}\right)\right) V_{\bar{M}, H-t-1}^{\pi}\left(s^{\prime}\right) d s^{\prime}\right)^{2}
\end{array}
$$
Then the Simulation lemma ensures that
$$
\mathbb{E}_{s_{0}}\left[V_{\bar{M}}^{\pi}\left(s_{0}\right)-V_{M}^{\pi}\left(s_{0}\right)\right]^{2} \leq 2 H \sum_{t=0}^{H-1} \mathbb{E}_{s_{t}, a_{t} \sim p_{M, \pi}}\left[\bar{l}_{r}\left(s_{t}, a_{t}, \hat{M}\right)+\bar{l}_{T}\left(s_{t}, a_{t}, \hat{M}\right)\right]
$$

The results are asymptotically consistent and thus unbiased!

Define the empirical risk over the behavior distribution and weighted distribution as:
$$
\begin{aligned}
\widehat{R}_{\mu}(\widehat{M}) &=\frac{1}{n} \sum_{i=1}^{n} \sum_{t=0}^{H-1} \ell_{r}\left(s_{t}^{(i)}, a_{t}^{(i)}, r^{(i)}, \widehat{M}\right)+\ell_{T}\left(s_{t}^{(i)}, a_{t}^{(i)}, s_{t}^{(i)}, \widehat{M}\right) \\
\widehat{R}_{\pi, u}(\widehat{M}) &=\frac{1}{n} \sum_{i=1}^{n} \sum_{t=0}^{H-1} \frac{\mathbb{1}\left(a_{0: t}^{(i)}=\pi\right)}{\hat{u}_{0: t}}\left[\ell_{r}\left(s_{t}^{(i)}, a_{t}^{(i)}, r^{(i)}, \widehat{M}\right)+\ell_{T}\left(s_{t}^{(i)}, a_{t}^{(i)}, s_{t}^{(i)}, \widehat{M}\right)\right]
\end{aligned}
$$
where $n$ is the dataset size, $s_{t}^{(i)}$ is the state of the $t^{\text {th}}$ step in the $i^{\text {th} ~ \text { trajectory, and } \widehat{u}_{0: t}=}$ $\sum_{i=1}^{n} \frac{1\left(a_{0 i t}^{(i)}=\pi\right)}{n}$

The bound is based on the empirical estimate of the marginal probability $u_{0: t}$ and we are not required to know the behavior policy. Our method's independence of the behavior policy is a significant advantage over IS methods which are very susceptible to errors its estimation, as we discuss in appendix estimate than $\mu$ when $\mu$ is unknown. The third term is an empirical estimate of IPM, which we described in Theorem@ We use norm-1 RKHS functions and MMD distance in this theorem and our algorithm. There are similar but worse results for Wasserstein distance and total variation distance. $\mathcal{D}_{\delta}^{\mathcal{F}}$ measures how complex $\mathcal{F}$ is. It is obtained from concentration measures about empirical IPM estimators. The constant $\mathcal{C}_{n, \delta, t}^{\mathcal{M}}$ measures how complex the model class is and it is derived from traditional learning theory results.

__Results on the Cartpole and the HIV simulations__


<img src="http://drive.google.com/uc?export=view&id=1-WDtPIxBd3kc1gva6jpSbnVRCAKC8e6n" width=50%>





Note: this paper assumes that there are no unobserved counfounders
    
    

## Sodhil, Arbour and Dimmery (2020) Balanced Off-Policy Evaluation in General Action Spaces

**Paper**, [here](http://proceedings.mlr.press/v108/sondhi20a/sondhi20a.pdf). 

__Similar idea as the previous paper, but this time the distance is measured in a different way__

Balanced off-policy evaluation (B-OPE) is a simple method for estimating balancing importance sampling weights, and the central contribution of this work. B-OPE leverages classifier-based density ratio estimation (Sugiyama et al., 2012; Menon and Ong, 2016) to learn importance sampling ratios. Specifically, offpolicy evaluation using B-OPE consists of four steps:
1. Create a supervised learning problem using the concatenated proposed policy instances $\left(s, a^{\prime}\right)$ and observed policy instances $(s, a),$ as covariates and giving a label $(C)$ of 0 to the observed policy and 1 to the proposed policy.
2. Learn a classifier to distinguish between the observed and proposed policy.
3. Take the importance sampling ratio as $\hat{\rho}\left(a_{i}, s_{i}\right)=$ $\hat{p}\left(C=1 \mid a_{i,} s_{i}\right)$
$\hat{p}\left(C=0 \mid a_{i, s i}\right)$
4. Replace IPS weights with the B-OPE estimates in any OPE method which uses them.

Step three arrives at the importance sampler through an application of Bayes rule (Bickel et al., 2009 )
$$
\frac{P(C=1 \mid a, s)}{P(C=0 \mid a, s)}=\frac{\pi(a, s \mid C=1) P(C=1)}{\pi(a, s \mid C=0) P(C=0)}=\frac{\pi_{1}(a, s)}{\pi_{0}(a, s)}
$$
where $\frac{P(C=1)}{P(C=0)}=1$ by design. 



__In a short amount of time there has been a vast development__

__Papers taking into account unobserved confounding:__

Namkoong et al. (2020) Off-policy Policy Evaluation For Sequential Decisions Under Unobserved Confounding

**Paper**, [here](https://arxiv.org/pdf/2003.05623.pdf). 

Wang, Yan and Wang (2020) Provably Efficient Causal Reinforcement Learning with Confounded
Observational Data

**Paper**, [here](https://arxiv.org/abs/2006.12311). 


__Learning causally correct models from high dimensional data__

Rezende et al (2020) Causally Correct Partial Models for Reinforcement Learning


**Paper**, [here](https://arxiv.org/pdf/2002.02836.pdf).


* Shows that it is central to learn causally correct models for reinforcement learning from high dimensional spaces
* E.g. compress an image from Space invaders in a way to represent the relevant causes

__Learning an explicit causal graph (SCM) from the environment__

Herlau (2020) Causal variables from reinforcement learning using generalized Bellman equations

**Paper**, [here](https://arxiv.org/format/2010.15745).


__Show that a meta-reinforcement learning algorithm that performs well in different environment learns some implicit causal reasoning (cognitive sciences perspective)__

Dasgupta et al. (2020) Meta-reinforcement learning of causal strategies 

**Paper**, [here](http://metalearning.ml/2019/papers/metalearn2019-dasgupta.pdf)

Literature related to the topic:
    
    
[here](https://openreview.net/pdf?id=BJG0voC9YQ)
[here](http://gershmanlab.webfactional.com/pubs/RL_causal.pdf)
[here](http://proceedings.mlr.press/v97/oberst19a/oberst19a.pdf)
[here](https://arxiv.org/pdf/2006.02579.pdf)
[here](https://causalai.net/mdp-causal.pdf)
[here](http://proceedings.mlr.press/v48/jiang16.pdf)
[here](https://papers.nips.cc/paper/2018/file/980ecd059122ce2e50136bda65c25e07-Paper.pdf)
[here](https://arxiv.org/pdf/2003.05623.pdf)
[here](http://proceedings.mlr.press/v119/zhang20a/zhang20a.pdf)
[here](https://arxiv.org/abs/2006.12311)
[here](https://arxiv.org/pdf/2002.02836.pdf)


Most of this research is from 2020!



    

    

    
    

    

            
            

                

    
   
    
    

        
        

    

    
